# Part 1 : Index Construction

## Data Crawling

In [1]:
from googlesearch import search 
import tldextract  
import requests 
from bs4 import BeautifulSoup 
import csv   
from time import sleep 

def format_data(doc_id, query, website, content):
    return {
        "doc_id" : doc_id,
        "query" : query,
        "website" : website,
        "content" : content
    }

# to search 
queries = ["Computer Science", "IITs in India", "Cities of Chattisgarh"]

docs = []

#Loop for each query
for i in range(len(queries)):
    query = queries[i]
    # Loop for each search result
    count_files = 20
    urls = []
    for url in search(query,num=25, start= 1, stop=50, pause=2): 
        urls.append(url)
        
    for url in urls:
        print(url)
        
        # pdf url are not useful for us as we can not extract data 
        #from it and sometimes they can create problem for us that's why we
        #are removing them
        if url[-3:] == 'pdf':
            continue
        
        try:
            respons = requests.get(url, timeout=5) #Response for the current URL
        except:
            continue
        soup = BeautifulSoup(respons.content, 'html5lib')
        paras=soup.find_all('p')  #Finding all the paragraph within the response
        para = "" #Initializing a empty paragraph
        
        count = 2 #To insure that file must contains 2 paragraph
        flag = 0
        
        for p in paras:   # Loop though each paragraph in response and append the contnet of that paragraph to  "para" so that we can save it as a continue paragraph
            if p!="":
                para += p.get_text()
                count-=1
            if (count==0):
                flag=1
                break
                
        if(flag):  #Perfom writing to file onlw when there is something in para
            ext_url = tldextract.extract(url)  #parsing the URL 
            domin_name = ext_url.domain   #Get the domain name from the parsed URL so that we can use it as a fine name
            
            docs.append(format_data(doc_id = f'd{21-count_files}_q{i+1}',
                                            query = query,
                                            website=url,
                                            content=para
                                           )
                               )
            count_files-=1
        
        if count_files==0:
            break

https://www.britannica.com/science/computer-science
https://www.edx.org/course/subject/computer-science
https://www.khanacademy.org/computing/computer-science
https://www.coursera.org/browse/computer-science
https://www.internationalstudent.com/study-computer-science/what-is-computer-science/
https://www.topuniversities.com/courses/computer-science-information-systems/guide
https://www.youtube.com/watch/Tzl0ELY_TiM
https://www.youtube.com/watch?v=nDDRliaoswg
https://www.timeshighereducation.com/student/what-to-study/computer-science
https://www.timeshighereducation.com/student/subjects/what-can-you-do-computer-science-degree
https://arxiv.org/archive/cs
https://www.computerscience.org/
https://www.seas.harvard.edu/computer-science
https://engineering.buffalo.edu/computer-science-engineering.html
https://www.codecademy.com/learn/paths/computer-science
https://www.stevens.edu/schaefer-school-engineering-science/departments/computer-science
https://github.com/ossu/computer-science
https:/

## Inverted Index

Associate a collection of terms (lexicon) with the documents that contain those terms.
The data structure is much more dense than a Document Term Matrix.

In [2]:
# Collection of documents (corpus)

'''
import json

with open('docs.json') as json_file:
    docs = json.load(json_file)
'''

docs 

[{'doc_id': 'd1_q1',
  'query': 'Computer Science',
  'website': 'https://www.britannica.com/science/computer-science',
  'content': 'Our editors will review what you’ve submitted and determine whether to revise the article.Computer science,  the study of computers and computing, including their theoretical and algorithmic foundations, hardware and software, and their uses for processing information. The discipline of computer science includes the study of algorithms and data structures, computer and network design, modeling data and information processes, and artificial intelligence. Computer science draws some of its foundations from mathematics and engineering and therefore incorporates techniques from areas such as queueing theory, probability and statistics, and electronic circuit design. Computer science also makes heavy use of hypothesis testing and experimentation during the conceptualization, design, measurement, and refinement of new algorithms, information structures, and co

In [3]:
# Gather the set of all unique terms

unique_terms = {term for doc in docs for term in doc["content"].split()}
unique_terms

{'IITs)',
 'education,',
 '25,540,196.',
 'ranks',
 'CS',
 'a',
 'Entrance',
 'pursue',
 'district.',
 'enthusiast',
 'years',
 'Preparation',
 'technology-oriented',
 'Framework)',
 'provided',
 'Technology',
 'chairman.',
 'research,',
 'lives',
 'site,',
 'Admission',
 'amongst',
 'Science:',
 'failure',
 'Soviet',
 'programming',
 'QnA,',
 'terms',
 'c.g.',
 'Madhya',
 'CSE',
 'focused',
 'nature',
 'fundamentals,',
 'field.',
 'society.',
 'Executive',
 'Obviously,',
 'secret.Widely',
 'post,',
 'University.',
 'fundamentals',
 'cities.',
 'for',
 'workforce',
 'IITS',
 'national',
 'back',
 'approximately',
 'field',
 '25.5',
 'the',
 'Mandi,',
 'helps',
 'generous',
 'contribution',
 'but',
 'thinking',
 'higher',
 'Institutes',
 'website',
 'via',
 'can',
 'Rajasthan',
 '2015',
 'platforms',
 'Supporting',
 '2016.',
 'institute',
 'PIN',
 'improvements',
 'mean',
 'avenues',
 'will',
 'development',
 'amid',
 'overseas',
 'K-12',
 '135,190',
 '•',
 'Harvard,',
 '"Learning',
 'o

In [4]:
# Construct an inverted index
# here as a Python dictionary for ease of interpretability

# word stemming, stop word removing with nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize  
from nltk.stem import PorterStemmer
stop_words = set(stopwords.words("english"))

ps = PorterStemmer()

inverted_index = {}
for doc in docs:
    for term in word_tokenize(doc["content"]):
        if term not in stop_words:
            term = ps.stem(term)
            if term in inverted_index:
                    inverted_index[term].add(doc["doc_id"])
            else: inverted_index[term] = {doc["doc_id"]}

inverted_index

{'our': {'d10_q3', 'd1_q1', 'd9_q3'},
 'editor': {'d10_q3', 'd1_q1', 'd9_q3'},
 'review': {'d10_q3', 'd1_q1', 'd3_q2', 'd9_q3'},
 '’': {'d10_q1',
  'd10_q3',
  'd11_q1',
  'd15_q2',
  'd18_q1',
  'd19_q1',
  'd1_q1',
  'd2_q1',
  'd8_q1',
  'd9_q3'},
 'submit': {'d10_q3', 'd1_q1', 'd9_q3'},
 'determin': {'d10_q3', 'd1_q1', 'd9_q3'},
 'whether': {'d10_q3', 'd1_q1', 'd9_q3'},
 'revis': {'d10_q3', 'd1_q1', 'd9_q3'},
 'article.comput': {'d1_q1'},
 'scienc': {'d10_q1',
  'd11_q1',
  'd12_q1',
  'd13_q1',
  'd14_q1',
  'd16_q1',
  'd18_q1',
  'd19_q1',
  'd1_q1',
  'd2_q1',
  'd3_q1',
  'd7_q1',
  'd7_q2',
  'd8_q1',
  'd8_q2',
  'd9_q2'},
 ',': {'d10_q1',
  'd10_q3',
  'd12_q3',
  'd13_q1',
  'd14_q1',
  'd14_q2',
  'd15_q1',
  'd15_q2',
  'd16_q1',
  'd16_q3',
  'd17_q3',
  'd18_q1',
  'd18_q2',
  'd18_q3',
  'd19_q1',
  'd1_q1',
  'd1_q2',
  'd20_q2',
  'd2_q1',
  'd2_q3',
  'd3_q1',
  'd3_q2',
  'd3_q3',
  'd4_q2',
  'd4_q3',
  'd6_q1',
  'd6_q2',
  'd7_q3',
  'd8_q1',
  'd8_q2',
  'd9_q

In [5]:
# saving the file containing the document ids and corresponding contents (along with the webpage link)
import json

with open("./docs.json", "w") as f:
    f.write(json.dumps(docs, indent=4))

In [6]:
# word with thier frequency in documents

freq = {}
for word in inverted_index:
    if len(inverted_index[word]) in freq:
        freq[len(inverted_index[word])].append(word)
    else:
        freq[len(inverted_index[word])] = [word]
freq

{3: ['our',
  'editor',
  'submit',
  'determin',
  'whether',
  'revis',
  'includ',
  'hardwar',
  'softwar',
  'inform',
  'design',
  'make',
  'test',
  'world',
  'interact',
  'CS',
  'power',
  'page',
  'pleas',
  'contribut',
  'secur',
  'school',
  'fundament',
  'societi',
  'advanc',
  'other',
  'covid-19',
  'video',
  'code',
  'career',
  'websit',
  'candid',
  'offer',
  'depart',
  'build',
  'innov',
  'recogn',
  'along',
  'provid',
  'As',
  'A',
  'autonom',
  'govern',
  'act',
  'framework',
  'overse',
  'total',
  'seat',
  'exam',
  '1951',
  'mumbai',
  'kanpur',
  'novemb',
  'ministri',
  'resourc',
  'nirf',
  'rank',
  'madra',
  'articl',
  'here',
  'clear',
  'set',
  'form',
  '1945',
  'committe',
  'least',
  'intern',
  'popul',
  'toward',
  'right',
  'post',
  'ensur',
  'north',
  'largest'],
 4: ['review',
  'algorithm',
  'theori',
  'also',
  'new',
  'field',
  '?',
  "'s",
  'need',
  'creat',
  'find',
  '2021',
  'these',
  '2020',


In [7]:
available_freq = sorted(freq)
available_freq

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 16, 18, 33, 39]

In [8]:
mostly_used = []
count = 3
for i in range(len(available_freq)-1,-1,-1):
    for item in freq[available_freq[i]]:
        if count==0:
            break
        mostly_used.append(item)
        count-=1
    if count==0:
        break

mostly_used

['.', ',', '(']

In [9]:
least_used = []
count = 3
for i in range(0,len(available_freq)):
    for item in freq[available_freq[i]]:
        if count==0:
            break
        least_used.append(item)
        count-=1
    if count==0:
        break

least_used

['article.comput', 'theoret', 'disciplin']